# Signal

Various utilities to do signal filtering and quality checking.

In [ ]:
# Imports
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from eegclassify import main, load, clean, features, preprocess, transform
from eegwatch.devices.muse import CHANNELS_MUSE

%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 5]

N_SAMPLES_PLOT = 400

In [ ]:
%%javascript
document.title='erb-thesis/Signal - Jupyter'  // Set the document title to be able to track time spent working on the notebook with ActivityWatch

In [ ]:
# Pick the last recording
files = load._get_all_recording_files()
files = sorted(files)[-1:]

# Load the last files
df = load.load_eeg(files)
X = df.drop(columns=['timestamp']).to_numpy()
print(X.shape)

In [ ]:
ax = sns.lineplot(data=X[-N_SAMPLES_PLOT:, :], dashes=False)
ax.set_title("Raw signal (pre-filtering)")
ax.legend(labels=CHANNELS_MUSE);

In [ ]:
X_f = clean.filter(X)
ax = sns.lineplot(data=X_f[-N_SAMPLES_PLOT:, :], dashes=False)
ax.set_title("Filtered signal")
ax.legend(labels=CHANNELS_MUSE);

In [ ]:
# Check the stddev of each channel
# TODO: refactor signal check function to actually use this
X_check = X_f[-N_SAMPLES_PLOT:, :]
print("mean\t", np.mean(X_check, axis=0))
print("std\t", np.std(X_check, axis=0))
print("maxabs\t", np.max(np.abs(X_check), axis=0))

In [ ]:
# std below this value is good
std_thres = 30

# std below this value is perfect
std_perfect = 10

def check(X: np.ndarray) -> float:
    std = np.std(X, axis=0)
    return std
    
def score(std: float) -> float:
    qual = std_thres - np.clip(std, std_perfect, std_thres)
    qual = qual / (std_thres - std_perfect)
    return np.clip(qual, 0, 1)

assert score(std_thres) == 0
assert score(std_perfect) == 1

In [ ]:
# Plot signal quality as timeline plot
cmap = matplotlib.cm.get_cmap('RdYlGn')

chunk_len = 200
for i, channel in enumerate(CHANNELS_MUSE):
    for si in range(chunk_len, X_f.shape[0], chunk_len):
        begin = si - chunk_len
        end = si
        segment = X_f[list(range(begin, end)), i]
        quality = score(check(segment))
        chidx = len(CHANNELS_MUSE) - i - 1
        plt.barh(chidx, end-begin, left=begin, color=cmap(quality))

plt.yticks(range(len(CHANNELS_MUSE)), reversed(CHANNELS_MUSE))
plt.title("Signal quality")
plt.show()